In [2]:
# Import libraries 
import json
import pandas as pd
!pip install yelpapi
from yelpapi import YelpAPI

In [30]:
# Load API Credentials
with open('/Users/aharo/.secret/yelp_api.json') as f:
    login = json.load(f)

In [31]:
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [41]:
# Define the search parameters
location = "Denver, CO"
cuisine = "Pizza"

In [42]:
# Perform the initial API call to get the total number of results
initial_results = yelp_api.search_query(location=location, term=cuisine, limit=50)
total_results = initial_results['total']

In [43]:
# Set up the dataframe
df = pd.DataFrame(initial_results['businesses'])

In [46]:
# Specify the desired limit per batch (<= 1000)
batch_limit = 50

In [49]:
# Determine the number of batches required for pagination
n_batches = (total_results + batch_limit - 1) // batch_limit

In [50]:
# Retrieve the results in batches
for batch_offset in range(1, n_batches + 1):
    offset = batch_offset * batch_limit
    batch_results = yelp_api.search_query(location=location, term=cuisine, limit=batch_limit, offset=offset)
    df = pd.concat([df, pd.DataFrame(batch_results['businesses'])], ignore_index=True)

YelpAPIError: VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.

In [51]:
# Check for duplicates
df.drop_duplicates(subset='id', inplace=True)

In [52]:
# Print the resulting dataframe
print(df.head())

                       id                                 alias  \
0  n18mJPI6mQfBJEp8UrITyw               blue-pan-pizza-denver-4   
1  VFACYV_lGv9y4FV-_1m4Ig  brooklyns-finest-pizza-lowell-denver   
2  Yk84dALUw0TPPf9sMuu2hg                 redeemer-pizza-denver   
3  mmz8zURQXK4ijOYkK1qBTg                 pizzeria-lui-lakewood   
4  kI5oOBAGS-Vo4chwmgOVmQ                    cart-driver-denver   

                      name                                          image_url  \
0           Blue Pan Pizza  https://s3-media2.fl.yelpcdn.com/bphoto/EAkVmo...   
1  Brooklyn's Finest Pizza  https://s3-media4.fl.yelpcdn.com/bphoto/6cPY-Z...   
2           Redeemer Pizza  https://s3-media4.fl.yelpcdn.com/bphoto/tmZdbk...   
3             Pizzeria Lui  https://s3-media1.fl.yelpcdn.com/bphoto/eW3v8m...   
4              Cart-Driver  https://s3-media1.fl.yelpcdn.com/bphoto/YzE2Pj...   

   is_closed                                                url  review_count  \
0      False  https://www.yel

In [53]:
# Save the dataframe to a file
df.to_csv('yelp_results.csv', index=False)